In [1]:
#to open cred files
import os
import yaml
#cleaning and operations
import pandas as pd
import numpy as np
#to read translations
import json
import unidecode

In [2]:
#bigquery
import pandas_gbq
from google.cloud import bigquery

In [3]:
## got to service accounts in google cloud in project OAN-Nikarit and download json file
os.environ['GOOGLE_APPLICATION_CREDENTIALS']='/Users/daniel/OAN/credentials/bigquery_oan_nikarit.json'

In [4]:
#FIREBASE
import firebase_admin
#Firbease (pip install firebase_admin)
from firebase_admin import credentials
from firebase_admin import firestore

In [5]:
filename='/Users/daniel/OAN/credentials/firebase_oan-nikarit.json'

In [6]:
#OPEN CONECTION TO FIREBASE
cred = credentials.Certificate(filename)
firebase_admin.initialize_app(cred)

db = firestore.client()

## Accounting table

### GET DATA

In [50]:
#get all the collection
col_query = db.collection('Accounting').stream()

In [51]:
#save it in a list of dictionaries
accounts = []
for acc in col_query:
    accounts.append(acc.to_dict())

In [52]:
#convert to dataframe
accounting = pd.DataFrame(accounts)

In [53]:
accounting['amount']=pd.to_numeric(accounting['amount'])

In [54]:
#nulls. size and datatype
accounting.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1697 entries, 0 to 1696
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   vat             1480 non-null   float64
 1   type            1697 non-null   object 
 2   intervention    1697 non-null   object 
 3   project         1697 non-null   object 
 4   account_id      1697 non-null   object 
 5   origin          1697 non-null   object 
 6   user_in_charge  1697 non-null   object 
 7   concept         1697 non-null   object 
 8   amount          1697 non-null   float64
 9   target_id       1697 non-null   object 
 10  execution_date  1697 non-null   object 
 11  creation_date   1697 non-null   object 
 12  phase           536 non-null    object 
 13  vat_amount      1634 non-null   float64
 14  place           1697 non-null   object 
 15  id              1697 non-null   object 
 16  description     1655 non-null   object 
 17  images          1694 non-null   o

In [55]:
#nulls heatmap - import seaborn as sns
#sns.heatmap(accounting.isna())

### CLEAN DATA

In [56]:
#lets see how it shows
accounting.head()

,vat,type,intervention,project,account_id,origin,user_in_charge,concept,amount,target_id,execution_date,creation_date,phase,vat_amount,place,id,description,images,creator_user,code
0,21.0,income,w04441aFcU5b7pQm6Rd2,0DmODGTOEiM5lg9SGx0J,3deqBmrZ1iJkAwMmxSOp,script_dani,IjBxXuBshlfq2MUzwBSK,venta shopify,21.00,yfy4cPxkmFwIYVnoWD0A,2020-04-28T12:07:49.000Z,2020-08-26T18:02:55.591Z,None,3.644628,spain,01WVDHF0gSSg1PCFsosk,venta online: 2288996286563,[],z5m936GA0t3vHM28QKhR,899
1,21.0,income,w04441aFcU5b7pQm6Rd2,0DmODGTOEiM5lg9SGx0J,3deqBmrZ1iJkAwMmxSOp,script_dani,IjBxXuBshlfq2MUzwBSK,venta shopify,38.00,iA9Pzv2CImjItzwCaQv0,2020-04-28T19:50:24.000Z,2020-08-26T18:02:55.591Z,None,6.595041,spain,026RXV3hyl33ussoU7D9,venta online: 2290422218851,[],z5m936GA0t3vHM28QKhR,1049
2,0.0,expense,SEDR5KRqPQVF6qovZiVq,RaIH6dfSXt4V7vm6jmVE,zMHMNTkAt43M0MmmRxJP,contoan,uCF6bT532woZwE9vQOuv,Piezas bomba Vergnet,132.65,vJbbj1kPxkcdXJyBOf1l,2020-08-21T11:18:01.503Z,2020-08-21T11:18:01.503Z,Dck0Vf7yfdGlBct3qoLY,0.000000,spain,02LozVMZvW51bygJ5e3O,Parte pagada por OAN International de piezas d...,[],uCF6bT532woZwE9vQOuv,417
3,0.0,expense,I76Ijc7H9N2k2R6rD2A6,opunMdqGyXDlfQ9DYIbN,NBrV00EhZDEdek9AMn0I,g_forms,aa9EdPDy2MPW1XkqhFUn,Forfait appelle et internet,7.62,nDhZYStFpM8fcUyqfswT,2019-12-10T00:00:00.0Z,2020-01-13T13:03:22.0Z,N6vKBVjEDW8XYOVrFemj,0.000000,benin,03mQM7enVuGkVqFou77J,Forfait communication décembre,[{'title': '20191210_Forfait appelle et intern...,zkGzYwDye123JxUktjSd,57
4,0.0,expense,lr1JTln4iPZheT4jtqq1,0DmODGTOEiM5lg9SGx0J,LaGxECvtRmDm9cjDr0nL,g_forms,exFnSrTD3fykjKbfaAvt,Achat de materiaux,27.40,nK5pyBaYL4y2sNk1RaxK,2020-04-17T00:00:00.0Z,2020-05-04T08:43:15.0Z,llgPMnqS4P3HGEkMiClU,0.000000,benin,06UGwVcSgVOTOifpuL22,Achat de pneu,"[{'file': {}, 'download_url': 'https://firebas...",0TCh5qYzrJVtwD4Sus7p,177


#### convert execution to isostring

In [57]:
def clean_execution(x):
    if(len(x['execution_date'])<15):
        doc_id = x['id']
        exec_date=x['execution_date']+'T14:14:58.593Z'
        print(exec_date)
        print(len(x['execution_date']))
        #upload to firebase if whished
        #a=db.collection(u'Accounting').document(doc_id).update({'execution_date':exec_date})
        
        return exec_date
    else:
        return x['execution_date']
        

In [58]:
accounting['execution_date']=accounting.apply(lambda x: clean_execution(x) ,axis=1)

2020-09-30T14:14:58.593Z
10
2020-09-01T14:14:58.593Z
10
2020-10-15T14:14:58.593Z
10
2020-10-31T14:14:58.593Z
10


##### clean VAT and replace nan with 0

In [59]:
def clean_vat_amount(x):
    doc_id = x['id']
    amount=x['amount']
    origin = x['origin']
    if origin =='script_dani':
        if x['vat']>0:
            vat=21
        else:
            vat=0
        #a=db.collection(u'Accounting').document(doc_id).update({'vat':vat})
    else:
        vat=x['vat']
        if not(vat >=0):
            vat=0
    vat_amount=amount*(1-1/(1+vat/100))

    #a=db.collection(u'Accounting').document(doc_id).update({'vat_amount':vat_amount})

    return pd.Series({'vat':vat,'vat_amount':vat_amount})

In [60]:
accounting[['vat','vat_amount']] = accounting.apply(lambda x: clean_vat_amount(x),axis=1)

###### convert dates to datetime

In [61]:
#utc=True important if dates are in diferent formats
accounting['creation_date'] = pd.to_datetime(accounting['creation_date'],utc=True)
accounting['execution_date'] = pd.to_datetime(accounting['execution_date'],utc=True)

In [62]:
def getimages(x):
    if not (x):
        return ''
    elif(len(x)==0):
        return ''
    elif('download_url' in x[0].keys()):
        download_url = x[0]['download_url']
        if(download_url == None):
            return ''
        else:
            return download_url
    else:
        return ''

In [63]:
accounting['receipt']=accounting.images.apply(lambda x: getimages(x))

##### double check

In [64]:
accounting.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1697 entries, 0 to 1696
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   vat             1697 non-null   float64            
 1   type            1697 non-null   object             
 2   intervention    1697 non-null   object             
 3   project         1697 non-null   object             
 4   account_id      1697 non-null   object             
 5   origin          1697 non-null   object             
 6   user_in_charge  1697 non-null   object             
 7   concept         1697 non-null   object             
 8   amount          1697 non-null   float64            
 9   target_id       1697 non-null   object             
 10  execution_date  1697 non-null   datetime64[ns, UTC]
 11  creation_date   1697 non-null   datetime64[ns, UTC]
 12  phase           536 non-null    object             
 13  vat_amount      1697 non-null   f

### select columns

In [65]:
#and the order whe want to upload to bigquery
df = accounting[['concept', 'place', 'creation_date', 'vat', 'description', 'amount',
       'id', 'phase', 'creator_user', 'execution_date',
       'user_in_charge', 'origin', 'account_id', 'project', 'code', 'type',
       'intervention', 'target_id', 'vat_amount', 'receipt']]

## upload to BIGQUERY

In [66]:
client = bigquery.Client()

In [67]:
dataset_id = "{}.contoan".format(client.project)

In [68]:
dataset = bigquery.Dataset(dataset_id)

In [69]:
table_id = "{}.accounting".format(dataset.dataset_id)

pandas_gbq.to_gbq(df, table_id, project_id=client.project, if_exists='replace')

1it [00:07,  7.82s/it]


### SUBIR VENTAS

#### get data

In [7]:
col_query = db.collection('Nikarit_Sales').stream()

In [8]:
ventas = []
for acc in col_query:
    ventas.append(acc.to_dict())

In [9]:
df_ventas = pd.DataFrame(ventas)

### view

In [11]:
df_ventas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 680 entries, 0 to 679
Data columns (total 56 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   packHidra_total_price    680 non-null    int64  
 1   client_last_name         676 non-null    object 
 2   creation_date            680 non-null    object 
 3   client_gender            680 non-null    object 
 4   total_tax                680 non-null    float64
 5   packHidra_unit_price     680 non-null    object 
 6   balsamo_total_price      680 non-null    int64  
 7   crema_total_price        680 non-null    int64  
 8   subtotal_price           680 non-null    object 
 9   product_movement         56 non-null     object 
 10  client_email             654 non-null    object 
 11  packContigo_total_price  680 non-null    int64  
 12  taxes_included           680 non-null    bool   
 13  origin                   680 non-null    object 
 14  manteca_nb               6

#### transform data

In [12]:
cambios = ['subtotal_price','balsamo_unit_price','packContigo_unit_price','crema_unit_price','packHidra_unit_price',
          'manteca_unit_price']
for cambio in cambios:
    df_ventas[cambio]=pd.to_numeric(df_ventas[cambio])

In [13]:
df_ventas['creation_date']=pd.to_datetime(df_ventas['creation_date'],utc=True)
df_ventas['close_date']=pd.to_datetime(df_ventas['close_date'],utc=True)

In [14]:
#double check
df_ventas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 680 entries, 0 to 679
Data columns (total 56 columns):
 #   Column                   Non-Null Count  Dtype              
---  ------                   --------------  -----              
 0   packHidra_total_price    680 non-null    int64              
 1   client_last_name         676 non-null    object             
 2   creation_date            680 non-null    datetime64[ns, UTC]
 3   client_gender            680 non-null    object             
 4   total_tax                680 non-null    float64            
 5   packHidra_unit_price     680 non-null    float64            
 6   balsamo_total_price      680 non-null    int64              
 7   crema_total_price        680 non-null    int64              
 8   subtotal_price           680 non-null    float64            
 9   product_movement         56 non-null     object             
 10  client_email             654 non-null    object             
 11  packContigo_total_price  680 non

### METER MUJERES Y HOMBRES

In [15]:
#client_gender

In [16]:
#lista de mujeres
url = 'https://raw.githubusercontent.com/marcboquet/spanish-names/master/mujeres.csv'
df_mujeres = pd.read_csv(url, error_bad_lines=False)
l_mujeres = df_mujeres['nombre'].to_list()

In [17]:
l_mujeres = [str(nom).lower() for nom in l_mujeres]

In [18]:
#lista de hombres
url = 'https://raw.githubusercontent.com/marcboquet/spanish-names/master/hombres.csv'
df_hombres = pd.read_csv(url, error_bad_lines=False)
l_hombres = df_hombres['nombre'].to_list()

In [19]:
l_hombres = [nom.lower() for nom in l_hombres]

In [20]:
df_ventas['client_name'].value_counts()

Ana                17
Carmen             14
Cristina           14
Daniel             11
María              10
                   ..
Emilia Lahilla      1
Jordi               1
INMACULADA          1
FATIMA              1
Mª Concepción       1
Name: client_name, Length: 367, dtype: int64

In [21]:
#aplicar sexo
df_ventas['client_gender'] = df_ventas.apply(lambda x: 
                            'F' if unidecode.unidecode(
                                    str(x['client_name']).replace(" ", "").lower()) in l_mujeres
                            else 'M' if unidecode.unidecode(
                                str(x['client_name']).replace(" ", "").lower())  in l_hombres
                            else 'F' if unidecode.unidecode(
                                str(x['client_name']).split(" ")[0].lower())  in l_mujeres
                            else 'M' if unidecode.unidecode(
                                str(x['client_name']).split(" ")[0].lower())  in l_hombres

                            else 'N/A' , axis =1)

In [22]:
#ver que sale
df_ventas['client_gender'].value_counts() # A MEJORRAR

F      525
M      125
N/A     30
Name: client_gender, dtype: int64

In [23]:
client = bigquery.Client()
dataset_id = "{}.contoan".format(client.project)
dataset = bigquery.Dataset(dataset_id)

In [24]:
#subir a bigquery
table_id = "contoan.sales"

pandas_gbq.to_gbq(df_ventas, table_id, project_id=client.project, if_exists='replace')

1it [00:05,  5.05s/it]


## Cuentas

In [87]:
## CSV generado al exportar la pestaña cuentas de Diario2020
df_cuentas = pd.read_csv('/Users/daniel/OAN/Nikarit/nikaritds/clients_study/cuentas.csv',index_col=0)
cuentas = df_cuentas.to_dict()['NOMBRE']
df_cuentas = df_cuentas.reset_index()
df_cuentas = df_cuentas[['ID','NOMBRE','NUMERO','TIPO']]
df_cuentas.columns = ['id','name','number','type']

In [88]:
## JSON File de Alfonso
with open('/Users/daniel/Downloads/fr.json') as json_file:
    cuentas_tr = json.load(json_file)

In [89]:
df_cuentas['name_fr']=df_cuentas['name'].apply(lambda x: cuentas_tr[x])

In [90]:
table_id = "{}.accounts".format(dataset.dataset_id)

pandas_gbq.to_gbq(df_cuentas, table_id, project_id=client.project, if_exists='replace')

1it [00:04,  4.65s/it]


### PROYECTOS

In [ ]:
## CSV generado al exportar la pestaña proyectos de Diario2020
df_projectos = pd.read_csv('/Users/daniel/OAN/Nikarit/nikaritds/clients_study/projectos.csv',index_col=0)
projectos = df_projectos.to_dict()['TITULO']
df_projectos = df_projectos.reset_index()
df_projectos = df_projectos[['ID','TITULO','DESCRIPCION','COMITE','TIPO','PROYECTO ID','INTERVENCION ID']]
df_projectos.columns = ['id','name','description','department','type','project_id','intervention_id']

In [ ]:
table_id = "{}.projects".format(dataset.dataset_id)

pandas_gbq.to_gbq(df_projectos, table_id, project_id=client.project, if_exists='replace')

### USUARIOS

In [ ]:
## CSV generado al exportar la pestaña usuarios de Diario2020
df_usuarios =pd.read_csv('/Users/daniel/OAN/Nikarit/nikaritds/clients_study/usuarios.csv',index_col=0)
usuarios = df_usuarios.to_dict()['NOMBRE']
df_usuarios = df_usuarios.reset_index()
df_usuarios = df_usuarios[['ID','NOMBRE','CUENTA']]
df_usuarios.columns = ['id','name','cuenta']

In [ ]:
table_id = "{}.users".format(dataset.dataset_id)

pandas_gbq.to_gbq(df_usuarios, table_id, project_id=client.project, if_exists='replace')

##### september 2020 backup

In [67]:
table_id = "{}.accounting_202009".format(dataset.dataset_id)

pandas_gbq.to_gbq(df, table_id, project_id=client.project, if_exists='replace')

1it [00:04,  4.48s/it]


## get data

In [ ]:
sql = """
    SELECT *
    FROM `oan-nikarit.contoan.accounting`
    LIMIT 100
"""

# Run a Standard SQL query using the environment's default project
df = pd.read_gbq(sql, dialect='standard')